In [2]:
import sqlite3
import random
from datetime import datetime, timedelta

# 데이터베이스 연결
conn = sqlite3.connect('korean_game_dev_project.db')
cursor = conn.cursor()

# 테이블 생성
cursor.executescript('''
CREATE TABLE IF NOT EXISTS Users (
    UserID INTEGER PRIMARY KEY AUTOINCREMENT,
    Username TEXT UNIQUE NOT NULL,
    Email TEXT UNIQUE NOT NULL,
    PasswordHash TEXT NOT NULL,
    FirstName TEXT,
    LastName TEXT,
    CreatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS Organizations (
    OrgID INTEGER PRIMARY KEY AUTOINCREMENT,
    OrgName TEXT NOT NULL,
    CreatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS UserOrganization (
    UserID INTEGER,
    OrgID INTEGER,
    Role TEXT,
    PRIMARY KEY (UserID, OrgID),
    FOREIGN KEY (UserID) REFERENCES Users(UserID),
    FOREIGN KEY (OrgID) REFERENCES Organizations(OrgID)
);

CREATE TABLE IF NOT EXISTS Projects (
    ProjectID INTEGER PRIMARY KEY AUTOINCREMENT,
    OrgID INTEGER,
    ProjectName TEXT NOT NULL,
    Description TEXT,
    StartDate DATE,
    EndDate DATE,
    Status TEXT,
    CreatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (OrgID) REFERENCES Organizations(OrgID)
);

CREATE TABLE IF NOT EXISTS Tasks (
    TaskID INTEGER PRIMARY KEY AUTOINCREMENT,
    ProjectID INTEGER,
    AssigneeID INTEGER,
    TaskName TEXT NOT NULL,
    Description TEXT,
    Status TEXT,
    Priority TEXT,
    DueDate DATE,
    EstimatedHours REAL,
    CreatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (ProjectID) REFERENCES Projects(ProjectID),
    FOREIGN KEY (AssigneeID) REFERENCES Users(UserID)
);

CREATE TABLE IF NOT EXISTS Comments (
    CommentID INTEGER PRIMARY KEY AUTOINCREMENT,
    TaskID INTEGER,
    UserID INTEGER,
    Content TEXT,
    CreatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (TaskID) REFERENCES Tasks(TaskID),
    FOREIGN KEY (UserID) REFERENCES Users(UserID)
);

CREATE TABLE IF NOT EXISTS Tags (
    TagID INTEGER PRIMARY KEY AUTOINCREMENT,
    TagName TEXT UNIQUE NOT NULL
);

CREATE TABLE IF NOT EXISTS TaskTags (
    TaskID INTEGER,
    TagID INTEGER,
    PRIMARY KEY (TaskID, TagID),
    FOREIGN KEY (TaskID) REFERENCES Tasks(TaskID),
    FOREIGN KEY (TagID) REFERENCES Tags(TagID)
);
''')

# 샘플 데이터
users = [
    ("김개발", "kim@gamedev.com", "개발자"),
    ("이디자인", "lee@gamedev.com", "디자이너"),
    ("박매니저", "park@gamedev.com", "프로젝트 매니저"),
    ("최아트", "choi@gamedev.com", "아티스트"),
    ("정품질", "jung@gamedev.com", "QA 테스터"),
    ("한기획", "han@gamedev.com", "기획자"),
    ("오사운드", "oh@gamedev.com", "사운드 디자이너"),
    ("서네트워크", "seo@gamedev.com", "네트워크 엔지니어"),
    ("강마케팅", "kang@gamedev.com", "마케팅 매니저"),
    ("임애니메이션", "lim@gamedev.com", "애니메이터")
]

organizations = [
    "판타지 월드 스튜디오",
    "인디 게임 공방",
    "모바일 게임 메이커스",
    "VR 게임 랩",
    "클래식 아케이드 리바이벌"
]

projects = [
    ("판타지 RPG 대모험", "오픈 월드 판타지 롤플레잉 게임", "진행 중"),
    ("우주 전쟁: 은하계 수호자", "멀티플레이어 우주 전투 게임", "기획 중"),
    ("퍼즐 마스터: 시간의 미로", "시간 여행 테마의 2D 퍼즐 어드벤처", "진행 중"),
    ("모바일 농장 타이쿤", "캐주얼 농장 경영 시뮬레이션", "테스트 중"),
    ("VR 공포 체험: 유령의 집", "가상현실 공포 게임", "기획 중"),
    ("레트로 아케이드 컬렉션", "클래식 아케이드 게임 모음", "완료"),
    ("AI 배틀: 전략의 시대", "AI 기반 전략 시뮬레이션", "진행 중"),
    ("음악의 리듬: 비트 챌린지", "리듬 액션 게임", "테스트 중"),
    ("미스터리 탐정: 그림자의 비밀", "추리 어드벤처 게임", "기획 중"),
    ("메카 워리어: 로봇 전쟁", "로봇 커스터마이징 액션 게임", "진행 중")
]

tasks = [
    "전투 시스템 구현",
    "캐릭터 커스터마이징 UI 디자인",
    "주요 캐릭터 컨셉 아트 제작",
    "전투 시스템 밸런싱 및 테스트",
    "우주선 컨트롤 시스템 설계",
    "프로젝트 타임라인 작성",
    "첫 10개 퍼즐 레벨 디자인",
    "배경 음악 작곡",
    "네트워크 멀티플레이어 기능 구현",
    "마케팅 전략 수립",
    "캐릭터 애니메이션 작업",
    "버그 수정 및 최적화",
    "게임 튜토리얼 제작",
    "로딩 화면 디자인",
    "NPC 대화 시스템 구현"
]

comments = [
    "플레이어 레벨에 따른 전투 난이도 조절이 필요해 보입니다.",
    "패리 시스템에서 버그를 발견했습니다. 상세 보고서를 작성하겠습니다.",
    "캐릭터 생성을 빠르게 할 수 있는 프리셋 추가를 고려해보면 어떨까요?",
    "주요 빌런의 컨셉이 훌륭합니다. 조금 더 위협적인 요소를 추가할 수 있을까요?",
    "독특한 컨트롤 방식에 대한 브레인스토밍 세션을 가져보는 게 어떨까요?",
    "휴가 시즌을 고려하여 타임라인을 조정해야 할 것 같습니다.",
    "이 퍼즐들 정말 재미있어 보입니다! 다음 주에 팀과 함께 플레이테스트를 진행해보죠.",
    "배경 음악이 게임의 분위기와 잘 어울립니다. 효과음 작업도 시작해볼까요?",
    "네트워크 지연 문제가 있습니다. 최적화가 필요해 보입니다.",
    "소셜 미디어 마케팅 캠페인 초안을 작성했습니다. 검토 부탁드립니다.",
    "주인공의 달리기 애니메이션이 부자연스러워 보입니다. 수정이 필요합니다.",
    "프레임 드랍 현상이 발생합니다. 렌더링 파이프라인 최적화가 필요해 보입니다.",
    "튜토리얼의 난이도가 너무 높습니다. 초보자를 위해 단계를 나누어 보는 것은 어떨까요?",
    "로딩 화면에 미니게임을 추가하면 재미있을 것 같아요.",
    "NPC의 대화가 반복적으로 느껴집니다. 다양성을 추가해 보면 어떨까요?"
]

tags = [
    "버그", "기능", "UI/UX", "아트", "디자인", "QA", "성능", "사운드", "네트워크", 
    "마케팅", "애니메이션", "최적화", "기획", "콘텐츠", "밸런싱"
]

# 데이터 삽입 함수
def insert_data():
    # Users 삽입
    for first_name, email, role in users:
        cursor.execute('''
        INSERT INTO Users (Username, Email, PasswordHash, FirstName, LastName)
        VALUES (?, ?, ?, ?, ?)
        ''', (f"{first_name}_{role}", email, f"hash_{first_name}", first_name, role))

    # Organizations 삽입
    for org in organizations:
        cursor.execute('INSERT INTO Organizations (OrgName) VALUES (?)', (org,))

    # UserOrganization 삽입
    for user_id in range(1, len(users) + 1):
        for org_id in range(1, len(organizations) + 1):
            if random.random() < 0.7:  # 70% 확률로 사용자를 조직에 할당
                cursor.execute('INSERT INTO UserOrganization (UserID, OrgID, Role) VALUES (?, ?, ?)',
                               (user_id, org_id, random.choice([u[2] for u in users])))

    # Projects 삽입
    for name, desc, status in projects:
        start_date = datetime.now() - timedelta(days=random.randint(0, 365))
        end_date = start_date + timedelta(days=random.randint(180, 730))
        cursor.execute('''
        INSERT INTO Projects (OrgID, ProjectName, Description, StartDate, EndDate, Status)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', (random.randint(1, len(organizations)), name, desc, start_date.date(), end_date.date(), status))

    # Tasks 삽입
    for _ in range(100):  # 100개의 작업 생성
        project_id = random.randint(1, len(projects))
        assignee_id = random.randint(1, len(users))
        task_name = random.choice(tasks)
        status = random.choice(["할 일", "진행 중", "검토 중", "완료"])
        priority = random.choice(["낮음", "보통", "높음", "긴급"])
        due_date = datetime.now() + timedelta(days=random.randint(1, 180))
        estimated_hours = random.randint(1, 100)
        
        cursor.execute('''
        INSERT INTO Tasks (ProjectID, AssigneeID, TaskName, Status, Priority, DueDate, EstimatedHours)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (project_id, assignee_id, task_name, status, priority, due_date.date(), estimated_hours))

    # Comments 삽입
    for _ in range(200):  # 200개의 댓글 생성
        task_id = random.randint(1, 100)
        user_id = random.randint(1, len(users))
        content = random.choice(comments)
        
        cursor.execute('INSERT INTO Comments (TaskID, UserID, Content) VALUES (?, ?, ?)',
                       (task_id, user_id, content))

    # Tags 삽입
    for tag in tags:
        cursor.execute('INSERT INTO Tags (TagName) VALUES (?)', (tag,))

    # TaskTags 삽입
    for task_id in range(1, 101):
        for _ in range(random.randint(1, 3)):  # 각 작업에 1-3개의 태그 할당
            tag_id = random.randint(1, len(tags))
            cursor.execute('INSERT OR IGNORE INTO TaskTags (TaskID, TagID) VALUES (?, ?)', (task_id, tag_id))

    conn.commit()

# 데이터 삽입 실행
insert_data()

# 연결 종료
conn.close()

print("한국어 게임 개발 프로젝트 관리 데이터베이스가 생성되었습니다.")

한국어 게임 개발 프로젝트 관리 데이터베이스가 생성되었습니다.
